In [2]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/5c/e9/ee572691a3fb05555bcde41826faad29ae4bc1fb07982e7f53d54a176879/scikit_learn-1.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 318.6 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 482.8 kB/s eta 0:00:00a 0:00:01
DEPRECATION: wfuzz 3.1.0 has a non-standard dependency specifier pyparsing>=2.4*; python_version >= "3.5". pip 23.3 will enforce this behaviour cha

In [1]:
import os
from collections import defaultdict

def count_files_with_same_extension(directory):
    extension_count = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(directory):
        for file in files:
            try:
                file_extension = os.path.splitext(file)[1]
                extension_count[root][file_extension] += 1
            except PermissionError:
                print(f"Permission denied: {os.path.join(root, file)}, skipping this file.")
            except Exception as e:
                print(f"Error occurred: {e}. Skipping this file.")
    return extension_count

def print_extension_counts(extension_count):
    for dir_path, extensions in extension_count.items():
        print(dir_path + ":")
        for ext, count in extensions.items():
            print(f"- with {ext} files: {count}")

def main():
    try:
        directory = 'data_v1'
        if not os.path.exists(directory):
            print("Invalid directory path. Please ensure the directory exists.")
            return
        extension_count = count_files_with_same_extension(directory)
        print_extension_counts(extension_count)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

data_v1:
- with .yaml files: 1
data_v1/test/labels:
- with .txt files: 535
data_v1/test/images:
- with .jpg files: 535
data_v1/valid:
- with .cache files: 1
data_v1/valid/labels:
- with .txt files: 1129
data_v1/valid/images:
- with .jpg files: 1129
data_v1/train:
- with .cache files: 1
data_v1/train/labels:
- with .txt files: 7677
data_v1/train/images:
- with .jpg files: 7677


In [3]:
import os
from collections import defaultdict

def count_files_with_same_extension(directory):
    extension_count = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(directory):
        for file in files:
            try:
                file_extension = os.path.splitext(file)[1]
                extension_count[root][file_extension] += 1
            except PermissionError:
                print(f"Permission denied: {os.path.join(root, file)}, skipping this file.")
            except Exception as e:
                print(f"Error occurred: {e}. Skipping this file.")
    return extension_count

def print_extension_counts(extension_count):
    for dir_path, extensions in extension_count.items():
        print(dir_path + ":")
        for ext, count in extensions.items():
            print(f"- with {ext} files: {count}")

def main():
    try:
        directory = 'filter-frame-similarity-groups'
        if not os.path.exists(directory):
            print("Invalid directory path. Please ensure the directory exists.")
            return
        extension_count = count_files_with_same_extension(directory)
        print_extension_counts(extension_count)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

filter-frame-similarity-groups:
- with .jpg files: 7477


In [5]:
import shutil
import glob

source_directory = 'filter-frame-similarity-groups/'
target_directory = 'filter-frame/'

import os
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

jpg_files = glob.glob(f'{source_directory}*.jpg')

for file in jpg_files:
    shutil.move(file, target_directory)

print(f"Moved {len(jpg_files)} .jpg files from {source_directory} to {target_directory}")

Moved 1707 .jpg files from filter-frame-similarity-groups/ to filter-frame/


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import os
import shutil
import numpy as np

# Define the directories
source_dir = '/media/ngoc/mydisk/ngoc/thesis/data/filter-frame'
target_dir = '/media/ngoc/mydisk/ngoc/thesis/data/filter-frame-similarity-groups'

# Create the target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Function to convert image to vector
def image_to_vector(image_path):
    image = Image.open(image_path)
    image = image.resize((100, 100)) # resizing the image
    return np.array(image).flatten()

# Read images and convert to vectors
image_vectors = []
image_paths = []
for filename in os.listdir(source_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(source_dir, filename)
        image_paths.append(image_path)
        image_vectors.append(image_to_vector(image_path))

# Calculate similarities and group images
threshold = 0.9 # similarity threshold
groups = []
while image_vectors:
    first_image = image_vectors.pop(0)
    group = [image_paths.pop(0)]
    i = 0
    while i < len(image_vectors):
        vector = image_vectors[i]
        similarity = cosine_similarity([first_image], [vector])
        if similarity >= threshold:
            group.append(image_paths.pop(i))
            image_vectors.pop(i)
        else:
            i += 1
    groups.append(group)

# Move the similar images to the target directory, retaining the first in each group
for group in groups:
    for image_path in group[1:]: # Skip the first image in the group
        shutil.move(image_path, os.path.join(target_dir, os.path.basename(image_path)))

print("Similarity filtering and grouping is complete.")

KeyboardInterrupt: 

cpu resources

In [8]:
from concurrent.futures import ProcessPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import os
import shutil
import numpy as np

# Define the directories
source_dir = 'filter-frame'
target_dir = 'filter-frame-similarity-groups'

# Create the target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Function to convert image to vector
def image_to_vector(image_path):
    image = Image.open(image_path)
    image = image.resize((100, 100)) # resizing the image
    return np.array(image).flatten()

# Read images and convert to vectors
image_vectors = []
image_paths = []
for filename in os.listdir(source_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(source_dir, filename)
        image_paths.append(image_path)
        image_vectors.append(image_to_vector(image_path))

# Function to find similar images
def find_similar_images(first_image, vectors, paths, threshold=0.9):
    group = []
    for i, vector in enumerate(vectors):
        similarity = cosine_similarity([first_image], [vector])
        if similarity >= threshold:
            group.append(paths[i])
    return group

# Parallelize finding similar images using all CPU cores
groups = []
with ProcessPoolExecutor() as executor:
    for i, result in enumerate(executor.map(find_similar_images, image_vectors, [image_vectors]*len(image_vectors), [image_paths]*len(image_paths))):
        groups.append([image_paths[i]] + result)

# Move the similar images to the target directory, retaining the first in each group
for group in groups:
    for image_path in group[1:]: # Skip the first image in the group
        shutil.move(image_path, os.path.join(target_dir, os.path.basename(image_path)))

print("Similarity filtering and grouping is complete.")


KeyboardInterrupt: 

reshape the vector to 2d, skip the first image in similarity group

In [6]:
from concurrent.futures import ProcessPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import os
import shutil
import numpy as np

source_dir = 'filter-frame'
target_dir = 'filter-frame-similarity-groups'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

def image_to_vector(image_path):
    image = Image.open(image_path)
    image = image.resize((100, 100))
    return np.array(image).flatten().reshape(1, -1) # Reshape the vector

image_vectors = []
image_paths = []
for filename in os.listdir(source_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(source_dir, filename)
        image_paths.append(image_path)
        image_vectors.append(image_to_vector(image_path))

def find_similar_images(first_image, vectors, paths, threshold=0.9):
    group = []
    for i, vector in enumerate(vectors):
        if i == 0: continue # Skip the first image itself
        similarity = cosine_similarity(first_image, vector) # Directly use the reshaped vector
        if similarity[0][0] >= threshold:
            group.append(paths[i])
    return group

groups = []
with ProcessPoolExecutor(max_workers=24) as executor:
    for i, result in enumerate(executor.map(find_similar_images, image_vectors, [image_vectors]*len(image_vectors), [image_paths]*len(image_paths))):
        groups.append([image_paths[i]] + result)

for group in groups:
    for image_path in group[1:]: # Start from the second image in the group
        if os.path.exists(image_path):
            try:
                shutil.move(image_path, os.path.join(target_dir, os.path.basename(image_path)))
                print(f"Successfully moved similar image: {image_path}") # Logging
            except Exception as e:
                print(f"Failed to move {image_path}: {str(e)}")
        else:
            print(f"Source file not found: {image_path}")

print("Similarity filtering and grouping is complete.")


KeyboardInterrupt: 

max cpu

In [2]:
from concurrent.futures import ProcessPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import os
import shutil
import numpy as np
import logging

source_dir = '../../filter-frame3'
target_dir = '../../filter-frame3-similarity-groups'
threshold = 0.9
image_size = (100, 100)
max_workers = 24

logging.basicConfig(level=logging.INFO)

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

def image_to_vector(image_path):
    try:
        with Image.open(image_path).convert('RGB') as image:
            image = image.resize(image_size)
            return np.array(image).flatten()
    except Exception as e:
        logging.error(f"Failed to process {image_path}: {str(e)}")
        return None

def process_images(image_paths):
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        vectors = list(executor.map(image_to_vector, image_paths))
    return [v for v in vectors if v is not None]

image_paths = [os.path.join(source_dir, filename) for filename in os.listdir(source_dir) if filename.endswith(".jpg")]
image_vectors = process_images(image_paths)

similarities = cosine_similarity(image_vectors)
processed_images = np.zeros(len(image_paths), dtype=bool)

for i in range(len(image_vectors)):
    if processed_images[i]:
        continue

    group = [j for j in range(len(image_vectors)) if similarities[i][j] >= threshold and not processed_images[j]]
    for j in group:
        processed_images[j] = True

    first_image = image_paths[i]
    for j in group[1:]:
        source_path = image_paths[j]
        target_path = os.path.join(target_dir, os.path.basename(source_path))
        logging.info(f"Moving from {source_path} to {target_path}")
        if os.path.exists(source_path):
            try:
                shutil.move(source_path, target_path)
            except Exception as e:
                logging.error(f"Failed to move {source_path}: {str(e)}")
        else:
            logging.error(f"Source file not found: {source_path}")

logging.info("Similarity filtering and grouping is complete.")

KeyboardInterrupt: 

efficient code

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import os
import shutil
import numpy as np
import logging

source_dir = '../../filter-frame3'
target_dir = '../../filter-frame3-similarity-groups'
threshold = 0.9
image_size = (100, 100)
max_workers = 24

logging.basicConfig(level=logging.INFO)

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

def image_to_vector(image_path):
    try:
        with Image.open(image_path).convert('RGB') as image:
            image = image.resize(image_size)
            return np.array(image).flatten()
    except Exception as e:
        logging.error(f"Failed to process {image_path}: {str(e)}")
        return None

def process_images(image_paths):
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        vectors = list(executor.map(image_to_vector, image_paths))
    return [v for v in vectors if v is not None]

image_paths = [os.path.join(source_dir, filename) for filename in os.listdir(source_dir) if filename.endswith(".jpg")]
image_vectors = process_images(image_paths)

similarities = cosine_similarity(image_vectors)
processed_images = np.zeros(len(image_paths), dtype=bool)

# Find similar images and move them
for i, similar_indices in enumerate((similarities >= threshold) & ~processed_images):
    group = np.where(similar_indices)[0]
    processed_images[group] = True

    for j in group[1:]:
        source_path = image_paths[j]
        target_path = os.path.join(target_dir, os.path.basename(source_path))
        logging.info(f"Moving from {source_path} to {target_path}")
        if os.path.exists(source_path):
            try:
                shutil.move(source_path, target_path)
            except Exception as e:
                logging.error(f"Failed to move {source_path}: {str(e)}")
        else:
            logging.error(f"Source file not found: {source_path}")

logging.info("Similarity filtering and grouping is complete.")

In [ ]:
import os
from collections import defaultdict

def count_files_with_same_extension(directory):
    extension_count = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(directory):
        for file in files:
            try:
                file_extension = os.path.splitext(file)[1]
                extension_count[root][file_extension] += 1
            except PermissionError:
                print(f"Permission denied: {os.path.join(root, file)}, skipping this file.")
            except Exception as e:
                print(f"Error occurred: {e}. Skipping this file.")
    return extension_count

def print_extension_counts(extension_count):
    for dir_path, extensions in extension_count.items():
        print(dir_path + ":")
        for ext, count in extensions.items():
            print(f"- with {ext} files: {count}")

def main():
    try:
        directory = '../../filter-frame3'
        if not os.path.exists(directory):
            print("Invalid directory path. Please ensure the directory exists.")
            return
        extension_count = count_files_with_same_extension(directory)
        print_extension_counts(extension_count)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import os
from collections import defaultdict

def count_files_with_same_extension(directory):
    extension_count = defaultdict(lambda: defaultdict(int))

    for root, dirs, files in os.walk(directory):
        for file in files:
            try:
                file_extension = os.path.splitext(file)[1]
                extension_count[root][file_extension] += 1
            except PermissionError:
                print(f"Permission denied: {os.path.join(root, file)}, skipping this file.")
            except Exception as e:
                print(f"Error occurred: {e}. Skipping this file.")
    return extension_count

def print_extension_counts(extension_count):
    for dir_path, extensions in extension_count.items():
        print(dir_path + ":")
        for ext, count in extensions.items():
            print(f"- with {ext} files: {count}")

def main():
    try:
        directory = '../../filter-frame3-similarity-groups'
        if not os.path.exists(directory):
            print("Invalid directory path. Please ensure the directory exists.")
            return
        extension_count = count_files_with_same_extension(directory)
        print_extension_counts(extension_count)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

gpu then cpu, if gpu is running out of memory 

In [3]:
import tensorflow as tf
from PIL import Image
import os
import shutil

# Define the directories
source_dir = 'filter-frame'
target_dir = 'filter-frame-similarity-groups'

# Create the target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Function to convert image to vector
def image_to_vector(image_path):
    image = Image.open(image_path)
    image = image.resize((100, 100)) # resizing the image
    image_array = tf.convert_to_tensor(image) # Convert to tensor
    image_array = tf.image.convert_image_dtype(image_array, tf.float32) # Normalize to [0, 1]
    return tf.reshape(image_array, [-1])

# Read images and convert to vectors
image_vectors = []
image_paths = []
for filename in os.listdir(source_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(source_dir, filename)
        image_paths.append(image_path)
        image_vectors.append(image_to_vector(image_path))

# Calculate similarities and group images
threshold = 0.9 # similarity threshold
groups = []
while image_vectors:
    first_image = image_vectors.pop(0)
    group = [image_paths.pop(0)]
    i = 0
    while i < len(image_vectors):
        vector = image_vectors[i]
        similarity = tf.reduce_mean(tf.keras.losses.cosine_similarity([first_image], [vector]))
        if similarity >= threshold:
            group.append(image_paths.pop(i))
            image_vectors.pop(i)
        else:
            i += 1
    groups.append(group)

# Move the similar images to the target directory, retaining the first in each group
for group in groups:
    for image_path in group[1:]: # Skip the first image in the group
        shutil.move(image_path, os.path.join(target_dir, os.path.basename(image_path)))

print("Similarity filtering and grouping is complete.")

KeyboardInterrupt: 

better code

In [1]:
import tensorflow as tf
import os
import shutil

source_dir = '../../filter-frame3'
target_dir = '../../filter-frame3-similarity-groups'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

def load_and_process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, (100, 100))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image_path, tf.reshape(image, [-1])

image_paths_ds = tf.data.Dataset.list_files(os.path.join(source_dir, "*.jpg"))
image_vectors_ds = image_paths_ds.map(load_and_process_image, num_parallel_calls=tf.data.AUTOTUNE)

image_vectors = []
image_paths = []
for path, vector in image_vectors_ds.as_numpy_iterator():
    image_vectors.append(vector)
    image_paths.append(path.decode())

threshold = 0.9
groups = []
while image_vectors:
    first_image = image_vectors.pop(0)
    group = [image_paths.pop(0)]
    i = 0
    while i < len(image_vectors):
        vector = image_vectors[i]
        similarity = tf.reduce_mean(tf.keras.losses.cosine_similarity([first_image], [vector]))
        if similarity >= threshold:
            group.append(image_paths.pop(i))
            image_vectors.pop(i)
        else:
            i += 1
    groups.append(group)

for group in groups:
    for image_path in group[1:]:
        shutil.move(image_path, os.path.join(target_dir, os.path.basename(image_path)))

print("Similarity filtering and grouping is complete.")

2023-08-17 09:57:38.307697: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-17 09:57:38.329934: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 09:57:38.720452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-17 09:57:39.137243: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
import os
import shutil

# Allow GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Case of having multiple GPUs, uncomment the following line to enable MirroredStrategy
# strategy = tf.distribute.MirroredStrategy()

source_dir = '../../filter-frame3'
target_dir = '../../filter-frame3-similarity-groups'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

def load_and_process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, (100, 100))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image_path, tf.reshape(image, [-1])

image_paths_ds = tf.data.Dataset.list_files(os.path.join(source_dir, "*.jpg"))

# Increase the number of parallel calls to utilize more workers
num_cores = os.cpu_count() # or another number that suits system
image_vectors_ds = image_paths_ds.map(load_and_process_image, num_parallel_calls=num_cores)

image_vectors = []
image_paths = []
for path, vector in image_vectors_ds.as_numpy_iterator():
    image_vectors.append(vector)
    image_paths.append(path.decode())

threshold = 0.9
groups = []
while image_vectors:
    first_image = image_vectors.pop(0)
    group = [image_paths.pop(0)]
    i = 0
    while i < len(image_vectors):
        vector = image_vectors[i]
        similarity = tf.reduce_mean(tf.keras.losses.cosine_similarity([first_image], [vector]))
        if similarity >= threshold:
            group.append(image_paths.pop(i))
            image_vectors.pop(i)
        else:
            i += 1
    groups.append(group)

for group in groups:
    for image_path in group[1:]:
        shutil.move(image_path, os.path.join(target_dir, os.path.basename(image_path)))

print("Similarity filtering and grouping is complete.")

2023-08-17 10:10:08.682380: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-17 10:10:08.705579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 10:10:09.109841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-17 10:10:09.538820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

Similarity filtering and grouping is complete.


cpu

In [6]:
import tensorflow as tf
import os
import shutil

# Configure TensorFlow to use all available CPU resources
num_threads = os.cpu_count() # Get the number of available CPU cores
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)

source_dir = 'filter-frame'
target_dir = 'filter-frame-similarity-groups'


if not os.path.exists(target_dir):
    os.makedirs(target_dir)

def load_and_process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, (100, 100))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image_path, tf.reshape(image, [-1])

image_paths_ds = tf.data.Dataset.list_files(os.path.join(source_dir, "*.jpg"))
image_vectors_ds = image_paths_ds.map(load_and_process_image, num_parallel_calls=tf.data.AUTOTUNE)

image_vectors = []
image_paths = []
for path, vector in image_vectors_ds.as_numpy_iterator():
    image_vectors.append(vector)
    image_paths.append(path.decode())

threshold = 0.9
groups = []
while image_vectors:
    first_image = image_vectors.pop(0)
    group = [image_paths.pop(0)]
    i = 0
    while i < len(image_vectors):
        vector = image_vectors[i]
        similarity = tf.reduce_mean(tf.keras.losses.cosine_similarity([first_image], [vector]))
        if similarity >= threshold:
            group.append(image_paths.pop(i))
            image_vectors.pop(i)
        else:
            i += 1
    groups.append(group)

for group in groups:
    for image_path in group[1:]:
        shutil.move(image_path, os.path.join(target_dir, os.path.basename(image_path)))

print("Similarity filtering and grouping is complete.")


RuntimeError: Inter op parallelism cannot be modified after initialization.